# DJ Runbook - Creating Nodes

This notebook performs a collection of basic requests to a running DJ server.

In [ ]:
import requests

DJ_PROTOCOL = "http"
DJ_HOST = "localhost"
DJ_PORT = 8000
DJ_URL = f"{DJ_PROTOCOL}://{DJ_HOST}:{DJ_PORT}"

Create some source nodes.

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "columns": {
            "payment_id": {"type": "INT"},
            "payment_type": {"type": "INT", "dimension": "payment_type"},
            "payment_amount": {"type": "FLOAT"},
            "customer_id": {"type": "INT", "dimension": "customers"},
            "account_type": {"type": "STR"},
        },
        "tables": [
            {
                "database_name": "postgres",
                "catalog": "test",
                "schema": "accounting",
                "table": "revenue",
            }
        ],
        "description": "A source table for revenue data",
        "mode": "published",
        "name": "revenue_source",
        "type": "source",
    },
)
response.json()

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "columns": {
            "id": {"type": "INT"},
            "account_type_name": {"type": "STR"},
            "account_type_classification": {"type": "INT"},
            "preferred_payment_method": {"type": "INT", "dimension": "payment_type"},
        },
        "tables": [
            {
                "database_name": "postgres",
                "catalog": "test",
                "schema": "accounting",
                "table": "account_type",
            }
        ],
        "description": "A source table for account type data",
        "mode": "published",
        "name": "account_type_table",
        "type": "source",
    },
)
response.json()

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "columns": {
            "id": {"type": "INT"},
            "payment_type_name": {"type": "STR"},
            "payment_type_classification": {"type": "INT"},
        },
        "tables": [
            {
                "database_name": "postgres",
                "catalog": "test",
                "schema": "accounting",
                "table": "payment_type",
            }
        ],
        "description": "A source table for different types of payments",
        "mode": "published",
        "name": "payment_type_table",
        "type": "source",
    },
)
response.json()

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "columns": {
            "customer_id": {"type": "INT"},
            "first_name": {"type": "STR"},
            "last_name": {"type": "STR"},
        },
        "tables": [
            {
                "database_name": "postgres",
                "catalog": "test",
                "schema": "accounting",
                "table": "customers",
            }
        ],
        "description": "A source table for customer data",
        "mode": "published",
        "name": "customers_table",
        "type": "source",
    },
)
response.json()

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "columns": {
            "customer_id": {"type": "INT"},
            "account_type": {"type": "STR", "dimension": "account_type"},
            "default_payment_type": {"type": "INT", "dimension": "payment_type"},
        },
        "tables": [
            {
                "database_name": "postgres",
                "catalog": "test",
                "schema": "accounting",
                "table": "customers",
            }
        ],
        "description": "A source table for customer default payment preference",
        "mode": "published",
        "name": "default_payment_account_table",
        "type": "source",
    },
)
response.json()

Create some dimension nodes.

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "description": "Payment type dimensions",
        "query": "SELECT id, payment_type_name, payment_type_classification FROM payment_type_table",
        "mode": "published",
        "name": "payment_type",
        "type": "dimension",
    },
)
response.json()

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "description": "Customer dimension",
        "query": "SELECT customer_id, first_name, last_name FROM customers_table",
        "mode": "published",
        "name": "customers",
        "type": "dimension",
    },
)
response.json()

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "description": "Account type dimension",
        "query": "SELECT id, account_type_name, account_type_classification FROM account_type_table",
        "mode": "published",
        "name": "account_type",
        "type": "dimension",
    },
)
response.json()

Create some transform nodes.

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "description": "Revenue data filtered to large payments only",
        "query": "SELECT payment_id, payment_amount, customer_id, account_type FROM revenue_source LEFT JOIN payment_type on payment_type = payment_type.id WHERE payment_amount > 1000000",
        "mode": "published",
        "name": "large_revenue_payments_only",
        "type": "transform",
    },
)
response.json()

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "description": "Revenue data filtered to large business payments only",
        "query": "SELECT payment_id, payment_amount, customer_id, account_type FROM revenue_source WHERE payment_amount > 1000000 AND account_type = 'BUSINESS'",
        "mode": "published",
        "name": "large_revenue_payments_and_business_only",
        "type": "transform",
    },
)
response.json()

Create some metrics nodes.

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "description": "Total company revenue",
        "query": "SELECT sum(payment_amount) as total_revenue FROM revenue_source",
        "mode": "published",
        "name": "total_revenue",
        "type": "metric",
    },
)
response.json()

In [ ]:
response = requests.post(
    f"{DJ_URL}/nodes/",
    json={
        "description": "Total number of account types",
        "query": "SELECT count(id) as num_accounts FROM account_type",
        "mode": "published",
        "name": "number_of_account_types",
        "type": "metric",
    },
)
response.json()